# Labelfree 2D
As the downstream tasks for the image compression. From the brightfield image to the fluorescent image.

In [1]:
# import modules:
from pathlib import Path
import pandas as pd
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
import numpy as np
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter
from pathlib import Path
import os
import sys
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import monai
import torch
import random
from datetime import datetime
import shutil

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
2023-10-16 10:32:56.671668: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# define funcs and hyper-parameters:

_SEED = 2023
np.random.seed(_SEED)
random.seed(_SEED)
os.environ["PYTHONHASHSEED"] = str(_SEED)

# firstly, use fine_tune_v12 to generate compressed images, and calculate metrics:
test_data_path = sorted(
    Path(
        "/mnt/eternus/users/Yu/project/data_compression/data/labelfree/FBL/holdout"
    ).glob("*IM.tiff")
)
compress_data_path = Path.cwd()
metrics = ["ms-ssim"]
models = ["bmshj2018-factorized"]

In [ ]:
for model in models:
    for metric in metrics:
        for image in test_data_path: 
            model_name= model+'_'+metric+'_'+'8_v12'
            path_encoded= compress_data_path/'prediction'/model_name/str(image.stem+'_encoded')
            path_encoded.parent.mkdir(exist_ok=True, parents = True)
            path_decoded= compress_data_path/'prediction'/model_name/str(image.stem+'_decoded.tiff')
            path_decoded.parent.mkdir(exist_ok=True, parents = True)
            if not path_decoded.is_file():
                !python3 ../../../codec.py encode {image} -o {path_encoded} --model {model} -q 8 -m {metric} --cuda --checkpoint { "/mnt/eternus/users/Yu/project/data_compression/experiment/paper_exp/2d/compression/bmshj2018-factorized_ms-ssim_8_v12/model.pth.tar"}
                # !python3 ../codec.py encode {image} -o {path_encoded} --model bmshj2018-factorized -q 8 -m {metric} --cuda
                !python3 ../../../codec.py decode {path_encoded} -o {path_decoded} --model {model} -q 8 -m {metric} --cuda --checkpoint {"/mnt/eternus/users/Yu/project/data_compression/experiment/paper_exp/2d/compression/bmshj2018-factorized_ms-ssim_8_v12/model.pth.tar"}

In [ ]:
# run the run_im2im inference code for compressed data.
!run_im2im --config_path /mnt/eternus/users/Yu/project/data_compression/experiment/paper_exp/2d/labelfree/compressed_config.yaml

In [ ]:
# run the run_im2im inference code for uncompressed data.
!run_im2im --config_path /mnt/eternus/users/Yu/project/data_compression/experiment/paper_exp/2d/labelfree/uncompressed_config.yaml